In [1]:
import torch 
from pyvi import ViTokenizer
from rage import Reranker

c:\Users\Admin\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin C:\Users\Admin\AppData\Roaming\Python\Python310\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Initialize Model

In [3]:
model= Reranker(model_name= "vinai/phobert-base-v2", torch_dtype= torch.float32, aggregation_hidden_states= False, 
                         strategy_pooling= "dense_first")

c:\Users\Admin\anaconda3\envs\llm\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model.to(device)

RerankerConfig(model_base: {'model_type_base': 'Reranker', 'model_name': 'vinai/phobert-base-v2', 'type_backbone': 'mlm', 'required_grad_base_model': True, 'aggregation_hidden_states': False, 'dropout': 0.1, 'quantization_config': None}, pooling: {'strategy_pooling': 'dense_first'})

In [5]:
model.load("ranker_phobert.pt", key= False)

In [6]:
model.summary_params()

trainable params: 135589633 || all params: 135589633 || trainable%: 100.0

In [9]:
model.summary()

+-----------------------------+-------------+------------------+
|         Layer (type)        |    Params   | Trainable params |
+-----------------------------+-------------+------------------+
|     model (RobertaModel)    | 134,998,272 |    134998272     |
|  pooling (PoolingStrategy)  |   590,592   |      590592      |
|        drp1 (Dropout)       |      0      |        0         |
| dropout_embedding (Dropout) |      0      |        0         |
|         fc (Linear)         |     769     |       769        |
+-----------------------------+-------------+------------------+


In [10]:
sentences= [
    "Tôi đang đi học", 
    "Bạn tên là gì?",
]

sentences= list(map(lambda x: ViTokenizer.tokenize(x), sentences))

# return_tensors= "pt" -> output is torch.Tensor
model.rank(sentences, batch_size= 1, return_tensors= "np", verbose= 1)

2/2 [==============================] - 1s 20ms/Sample


e:\rag_chatbot\src\rage\models\rerank\model.py:151: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return nn.Sigmoid()(torch.tensor(embedding))


array([0.5040935 , 0.49734432], dtype=float32)

In [11]:
model.push_to_hub("anti-ai/phobert-ranker-base")

pytorch_model.bin: 100%|██████████| 542M/542M [01:57<00:00, 4.60MB/s]   


'https://huggingface.co/anti-ai/phobert-ranker-base/tree/main/'

### Load model from Huggingface Hub

In [14]:
model= Reranker.from_pretrained('anti-ai/phobert-ranker-base')

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model

RerankerConfig(model_base: {'model_type_base': 'Reranker', 'model_name': 'vinai/phobert-base-v2', 'type_backbone': 'mlm', 'required_grad_base_model': True, 'aggregation_hidden_states': False, 'dropout': 0.1, 'quantization_config': None}, pooling: {'strategy_pooling': 'dense_first'})